### Imports

In [ ]:
import cv2
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert
import torch

### Image preprocessing

In [ ]:
test_image = 'test_image.jpg'

# convert to grayscale
img = Image.open(test_image).convert('L')

# invert image
img = invert(img)

# increase contrast of image
img = ImageEnhance.Contrast(img).enhance(2)

# make all pixels below 200 black
img = img.point(lambda p: p > 220 and 255)

# smoothen image
img = img.filter(ImageFilter.SMOOTH)

# show image
img.show()


### Inference

In [ ]:
# model path
model_path = 'best.pt'

# load model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
model.eval()

# get results
results = model(img)

# print results
print(results.pandas().xyxy)

# show results
results.show()

# save image
results.save('test_image_result.jpg')

### Line detection

In [ ]:
import numpy as np
# Remove pixels in bounding boxing from image
def remove_pixels(image, results):
    # get bounding boxes
    boxes = results.xyxy[0]

    # convert image to numpy array
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # remove pixels in bounding boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box[0:4])  # Convert coordinates to integers
        image[y1:y2, x1:x2] = 0

    # convert image back to PIL image
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    return image

# remove pixels in bounding boxes
image = remove_pixels(img, results)

# # show image
# image.show()

In [ ]:
# Run line detection on image
import cv2

# convert image to numpy array
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# find edges in image
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# find lines in image
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=10, minLineLength=50, maxLineGap=10)

print(len(lines))

# draw lines on image
for points in lines:
    x1, y1, x2, y2 = points[0]
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# convert image back to PIL image
image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# show image
image.show()